In [1]:
import sys

sys.path.append('../src')

In [1]:
def check_cml_env():
    return 'PREPROD'


import os
import warnings
from typing import List, Union
from ruyaml import YAML
from pathlib import Path
from dynaconf.utils import object_merge
from dynaconf import Dynaconf, Validator
from dynaconf.utils.boxing import DynaBox


ENCODING = 'utf-8'
ENV = check_cml_env()
ENV_VARS_SECTION = 'env_variables'
YAML_PATH_MAPPING = {
    'UAT': '../config/uat.yaml',
    'PROD': '../config/prod.yaml',
    'PREPROD': '../config/preprod.yaml',
    'NO_CML': '../config/no_cml.yaml'
}


class Config(Dynaconf):
    ALLOWED_EXTENSIONS = ['.yaml', '.yml']

    def __init__(self, 
                 settings_file: str = None, 
                 load_env_vars: bool = True,
                 validators: List[Validator] = None,
                 env_vars_key: str = ENV_VARS_SECTION,
                 apply_default_on_none: bool = True,
                 validate_on_update: Union[bool, str] = False, 
                 ):
        if settings_file is None:
            settings_file = YAML_PATH_MAPPING.get(ENV)

        self.validate_extension(settings_file)

        # TODO: Add in validators for certain sections in the config yaml that we think should always be there
        super().__init__(
            settings_file = settings_file, 
            validators = validators,
            apply_default_on_none = apply_default_on_none,
            core_loaders = ['YAML'],
            loaders = [], # TODO: replace with custom YAML loader that preserves comments
            validate_on_update = validate_on_update,
        )
        
        if load_env_vars:
            self.set_env_vars(env_vars_key)

    def validate_extension(self, filepath: str):
        _, ext = os.path.splitext(filepath)
        if ext not in self.ALLOWED_EXTENSIONS:
            raise NotImplementedError(f'File type not supported for reading: {ext}')
        
    def set_env_vars(self, env_vars_key: str = ENV_VARS_SECTION):
        env_variables = getattr(self, env_vars_key, None)
        if env_variables is None:
            warnings.warn(
                f'Skip the setting of environment variables as {env_vars_key} section is not found in {self.settings_file}'
            )
            return

        for key, value in env_variables.items():
            os.environ[key] = str(value)

    def reload(self, settings_module = None, **kwargs):
        super().configure(settings_module, **kwargs)

    def to_yaml(self, filepath: str = None):
        if filepath is None:
            filepath = self.settings_file

        self.validate_extension(filepath)
        data = DynaBox(self._wrapped.as_dict(), box_settings = {}).to_dict()
        # if Path(filepath).exists():
        #     with open(filepath, encoding = ENCODING) as f:
        #         object_merge(YAML().load(f), data)
        
        with open(filepath, 'wb') as f:
            YAML().dump(data, f)

    # TODO: monkeypatch YAML loader in dynaconf loaders so that it uses ruamel yaml which lets us preserve comments and order
    # TODO: monkeypatch YAML loader such that it allows us to read from HDFS also (low priority)

    # TODO: reload and write to file should be user managed rather than by config as it is expensive to reload and write to file each time we update an attribute
    # TODO: Fix the uppercase issue where dynaconf will uppercase all the first level attributes (known limitation that will be fixed in 4.0.0, live with it for now)
    # TODO: known limitations, first level have to be uppercase

In [2]:
config = Config(validators = [Validator('ENV_VARIABLES', must_exist = True)])

In [45]:
config.as_dict()

{'CREDENTIALS': {'lan_id': 'a5123456',
  'path': '/user/a5123456/credentials.pwd'},
 'ENV_VARIABLES': {'test_env_variable': 1},
 'EXTRA': None}

In [37]:
config.to_yaml()

In [4]:
config.reload()

In [5]:
config.as_dict()

{'CREDENTIALS': {'lan_id': 'a5123456',
  'path': '/user/a5123456/credentials.pwd'},
 'ENV_VARIABLES': {'test_env_variable': 1}}

In [276]:
config.set('bye', 1)

In [280]:
config.update({'bye': 3})

In [281]:
config.bye = 2

In [283]:
config.as_dict()

{'ENV_VARS_KEY': 'env_variables',
 'CREDENTIALS': {'lan_id': 'a5123456',
  'path': '/user/a5123456/credentials.pwd'},
 'ENV_VARIABLES': {'test_env_variable': 1},
 'BYE': 4}

In [282]:
config['bye'] = 4

In [7]:
from dynaconf import loaders
from dynaconf.loaders import yaml_loader

loaders.write('../config/preprod.yaml', config.as_dict())

In [6]:
from ruyaml import YAML

with open('../config/preprod.yaml', encoding = 'utf-8') as f:
    test = YAML().load(f)

In [47]:
test['extra'] ='extra'

In [48]:
test

ordereddict([('CREDENTIALS', ordereddict([('lan_id', 'a5123456'), ('path', '/user/a5123456/credentials.pwd')])), ('ENV_VARIABLES', ordereddict([('test_env_variable', 1)])), ('extra', 'extra')])

In [49]:
with open('../config/preprod.yaml', 'wb') as f:
    YAML().dump(test, f)

In [51]:
config.credentials.lan_id

<Box: {'lan_id': 'a5123456', 'path': '/user/a5123456/credentials.pwd'}>